In [1]:
import pandas as pd
import fundamentus as fund
import yfinance as yf

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [2]:
def filterTheDict(dictObj, callback):
    newDict = dict()
    # Iterate over all the items in dictionary
    for (key, value) in dictObj.items():
        # Check if item satisfies the given condition then add to new dict
        if callback((key, value)):
            newDict[key] = value
    return newDict

In [3]:
%%time

lista = fund.get_data()

100%|██████████| 426/426 [03:47<00:00,  1.88it/s]Wall time: 3min 47s



In [4]:
%%time 

lista.groupby('setor')['codigo'].count().reset_index(name='qtd acoes').sort_values(['qtd acoes'], ascending=False) 

Wall time: 4 ms


,setor,qtd acoes
12,Energia Elétrica,67
18,Intermediários Financeiros,37
34,"Tecidos, Vestuário e Calçados",28
8,Construção Civil,25
6,Comércio,24
14,Exploração de Imóveis,16
33,Siderurgia e Metalurgia,16
24,Máquinas e Equipamentos,15
27,Previdência e Seguros,15
36,Transporte,13


In [5]:
%%time 

stock_list = lista[['setor', 'codigo', 'cotacao', 'pl', 'roe', 'div_yield', 'lpa']].sort_values(['setor'], ascending=False).reset_index()
stock_list['y_codigo'] = (stock_list.codigo + '.SA')

Wall time: 27 ms


In [6]:
%%time 

stock_data = yf.download(list(stock_list['y_codigo']),  period = "2y")
stock_closed = stock_data['Close']

[*********************100%***********************]  426 of 426 completed
Wall time: 18.1 s


In [7]:
%%time 

mu = mean_historical_return(stock_closed)
S = CovarianceShrinkage(stock_closed).ledoit_wolf()

Wall time: 1.1 s


In [8]:
%%time 

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

Expected annual return: 158.6%
Annual volatility: 62.1%
Sharpe Ratio: 2.52
Wall time: 2.32 s


(1.5861335552137, 0.6207610945622705, 2.522924791731767)

In [9]:
# %%time 

# latest_prices = get_latest_prices(stock_closed)
# da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1200)
# allocation, leftover = da.lp_portfolio()
# print(allocation)
# print(leftover)

In [10]:
%%time 

chosen_stocks = filterTheDict(cleaned_weights, lambda elem: elem[1] > 0)
chosen_stocks

Wall time: 0 ns


{'AFLT3.SA': 0.00295,
 'AGRO3.SA': 0.00134,
 'ALPA3.SA': 0.00293,
 'ALPA4.SA': 0.0039,
 'ANIM3.SA': 0.00107,
 'APER3.SA': 0.00218,
 'ARZZ3.SA': 0.00095,
 'ATOM3.SA': 0.00182,
 'B3SA3.SA': 0.00244,
 'BALM3.SA': 0.00209,
 'BALM4.SA': 0.00059,
 'BAUH4.SA': 0.0079,
 'BAZA3.SA': 0.00134,
 'BEEF3.SA': 0.00157,
 'BEES4.SA': 0.00025,
 'BGIP3.SA': 0.00031,
 'BIDI3.SA': 0.00384,
 'BIDI4.SA': 0.01586,
 'BIOM3.SA': 0.00422,
 'BMEB3.SA': 0.00469,
 'BMEB4.SA': 0.004,
 'BMIN3.SA': 0.00019,
 'BNBR3.SA': 0.00208,
 'BPAC3.SA': 0.00396,
 'BPAC5.SA': 0.00712,
 'BPAN4.SA': 0.00921,
 'BRAP3.SA': 0.00369,
 'BRAP4.SA': 0.00366,
 'BRGE11.SA': 0.00291,
 'BRGE12.SA': 0.00301,
 'BRGE3.SA': 0.00311,
 'BRGE5.SA': 0.00832,
 'BRGE6.SA': 0.00262,
 'BRGE8.SA': 0.00264,
 'BRIV3.SA': 0.00173,
 'BRIV4.SA': 0.00066,
 'BSEV3.SA': 0.00543,
 'BSLI3.SA': 0.02731,
 'BSLI4.SA': 0.01753,
 'BTOW3.SA': 0.00157,
 'BTTL3.SA': 0.00383,
 'CAML3.SA': 0.00193,
 'CARD3.SA': 0.00322,
 'CASN3.SA': 0.00287,
 'CEBR3.SA': 0.01015,
 'CEBR5.SA':

In [13]:
len(chosen_stocks.keys())

233

In [18]:
stocks_chosen = pd.DataFrame.from_dict(cleaned_weights, orient='index', columns=['distribuicao'])

In [25]:
stocks_chosen.filter(like='AGRO3.SA', axis=0)

,distribuicao
AGRO3.SA,0.00134


In [20]:
acoes = pd.merge(left=stock_list, right=stocks_chosen, how='inner', left_on='y_codigo', right_index=True)

In [21]:
acoes

,index,setor,codigo,cotacao,pl,roe,div_yield,lpa,y_codigo,distribuicao
0,15,Água e Saneamento,AMBP3,23.15,0.0,0.0,0.0,0.0,AMBP3.SA,0.00000
1,138,Água e Saneamento,CSMG3,14.65,6.82,0.126,0.204,2.15,CSMG3.SA,0.00000
2,353,Água e Saneamento,SAPR3,4.13,6.26,0.14400000000000002,0.045,0.66,SAPR3.SA,0.00000
3,354,Água e Saneamento,SAPR4,4.13,6.26,0.14400000000000002,0.049,0.66,SAPR4.SA,0.00000
4,355,Água e Saneamento,SBSP3,38.29,21.83,0.055,0.036000000000000004,1.75,SBSP3.SA,0.00000
...,...,...,...,...,...,...,...,...,...,...
421,367,Alimentos Processados,SMTO3,31.16,12.79,0.214,0.017,2.44,SMTO3.SA,0.00201
422,363,Agropecuária,SLCE3,42.86,20.94,0.14400000000000002,0.023,2.05,SLCE3.SA,0.00378
423,4,Agropecuária,AGRO3,22.85,9.61,0.17,0.023,2.38,AGRO3.SA,0.00134
424,201,Agropecuária,FRTA3,5.0,3.43,0.045,0.0,1.46,FRTA3.SA,0.00000


In [11]:
# import numpy as np
# prices = np.isnan(latest_prices)

In [12]:
# prices[prices == True]